In [1]:
import pandas as pd
from datasets import Dataset

In [2]:
# Reading the file
f = open("en2cn-2k.en2nen2cn", "r", encoding = 'utf-8')
text = f.read()

# Removing last instance after splitting as it is empty string
text = text.split('\n')[:-1]

# Creating the pandas dataframe
data = [[text[i], text[i+1]] for i in list(range(0, 6000, 3))]
df = pd.DataFrame(data, columns = ['Informal text', 'Formal text'])
df.head()

dataset = Dataset.from_pandas(df)

# Split the dataset into training and test sets
dataset = dataset.train_test_split(test_size=0.1)

In [3]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

def tokenize_function(examples):
    # This function will now correctly handle the input as expected
    model_inputs = tokenizer(examples['Informal text'], padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(examples['Formal text'], padding="max_length", truncation=True, max_length=512).input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss
